In [19]:
import modin.pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import plotly as pl



In [7]:
train_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/train.parquet')
transactions_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/transactions.parquet')
stores_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/stores.parquet')
holidays_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/holidays_events.parquet')
oil_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/oil.parquet')


Perhaps you already have a cluster running?
Hosting the HTTP server on port 45941 instead


In [8]:
print(train_df['date'].min())
print(train_df['date'].max())

2013-01-01
2017-08-15


In [9]:
train_df.nlargest(20, 'sales')

,id,date,store_nbr,family,sales,onpromotion
2163723,2163723,2016-05-02,2,GROCERY I,124717.000,59
2445984,2445984,2016-10-07,39,MEATS,89576.360,0
2144154,2144154,2016-04-21,20,GROCERY I,87438.516,53
2139699,2139699,2016-04-18,45,GROCERY I,76090.000,38
2153031,2153031,2016-04-26,2,GROCERY I,63434.000,30
2145045,2145045,2016-04-21,45,GROCERY I,53874.000,44
562596,562596,2013-11-12,44,GROCERY I,46271.000,0
1257246,1257246,2014-12-08,35,GROCERY I,45361.000,3
1916586,1916586,2015-12-14,35,GROCERY I,40351.460,7
2760297,2760297,2017-04-02,9,GROCERY I,38422.625,73


In [10]:
# check data quality of train_df
train_df.isnull().sum()


id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [11]:

store_sales = train_df.merge(stores_df, on='store_nbr', how='left')

In [12]:
# do a sum of sales grouped by cluster and date
cluster_sales = store_sales.groupby(['cluster', 'date']).agg({'sales': 'sum'}).reset_index()

In [13]:

cluster_sales['date'] = pd.to_datetime(cluster_sales['date'])
cluster_sales['week'] = cluster_sales['date'].dt.isocalendar().week



Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [14]:

clusters = cluster_sales['cluster'].unique()

In [15]:
#group by week and cluster
weekly_sales = cluster_sales.groupby(['cluster', 'week']).agg({'sales': 'sum'}).reset_index()

In [18]:
cluster_1 = weekly_sales[weekly_sales['cluster'] == 1]

fig = go.Figure()

fig.add_trace(go.Scatter(x=cluster_1['week'], 
                         y=cluster_1['sales'], 
                         mode='lines+markers', 
                         name='Cluster 1'))


fig.update_layout(title='Sales by Week for Cluster 1',
                  xaxis_title='Week',
                  yaxis_title='Sales')

# Show the plot
fig.show()


In [20]:
fig = go.Figure()

clusters = weekly_sales['cluster'].unique()

for cluster in clusters:
    cluster_data = weekly_sales[weekly_sales['cluster'] == cluster]
    fig.add_trace(go.Scatter(x=cluster_data['week'], 
                             y=cluster_data['sales'], 
                             mode='lines+markers', 
                             name=f'Cluster {cluster}'))
    
fig.update_layout(title='Sales by Week for All Clusters',
                  xaxis_title='Week',
                  yaxis_title='Sales')

fig.show()
